## Q1 converting Keras model to TF-Lite

In [9]:
from tensorflow import keras
import tensorflow as tf

In [8]:
model = keras.models.load_model('dogs_cats_10_0.687.h5')

2021-12-01 21:20:32.908936: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('dogs_cats.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

2021-12-01 21:21:43.800832: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /var/folders/sv/0_vx_sd17318_hf3rq61b32h0000gp/T/tmp__65rls7/assets


2021-12-01 21:21:45.213380: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2021-12-01 21:21:45.213396: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2021-12-01 21:21:45.216833: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /var/folders/sv/0_vx_sd17318_hf3rq61b32h0000gp/T/tmp__65rls7
2021-12-01 21:21:45.220769: I tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2021-12-01 21:21:45.220785: I tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: /var/folders/sv/0_vx_sd17318_hf3rq61b32h0000gp/T/tmp__65rls7
2021-12-01 21:21:45.236490: I tensorflow/cc/saved_model/loader.cc:210] Restoring SavedModel bundle.
2021-12-01 21:21:45.428126: I tensorflow/cc/saved_model/loader.cc:194] Running initialization op on SavedModel bundle at path: /var/folders/sv/0_vx_sd17318_hf3rq61b32h0000gp/T/tmp__65rls7
2021-12

In [13]:
!ls -lha | grep tflite

-rw-r--r--   1 abarabas  staff    43M Dec  1 21:21 dogs_cats.tflite


## Q2 output index

In [14]:
import tensorflow.lite as tflite

In [15]:
interpreter = tflite.Interpreter(model_path='dogs_cats.tflite')
interpreter.allocate_tensors()

In [16]:
interpreter.get_input_details()

[{'name': 'serving_default_conv2d_1_input:0',
  'index': 0,
  'shape': array([  1, 150, 150,   3], dtype=int32),
  'shape_signature': array([ -1, 150, 150,   3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [17]:
interpreter.get_output_details()

[{'name': 'StatefulPartitionedCall:0',
  'index': 13,
  'shape': array([1, 1], dtype=int32),
  'shape_signature': array([-1,  1], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [18]:
interpreter.get_output_details()[0]['index']

13

In [19]:
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

## Q3 image preprocessing

In [21]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [23]:
! curl https://upload.wikimedia.org/wikipedia/commons/9/9a/Pug_600.jpg -O Pug_600.jpg

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 99389  100 99389    0     0   315k      0 --:--:-- --:--:-- --:--:--  315k
curl: (6) Could not resolve host: Pug_600.jpg


In [100]:
def preprocess_image(url):
    raw_img = download_image(url)
    img = prepare_image(raw_img, target_size)
    x = np.array(img, dtype='float32')
    # rescaling
    return x/255

In [101]:
x = preprocess_image("https://upload.wikimedia.org/wikipedia/commons/9/9a/Pug_600.jpg")
# the R component of the first pixel
x[0][0][0]

0.7058824

## Q4 applying the model

In [102]:
import numpy as np
def predict(x):
    X = np.array([x])
    interpreter.set_tensor(input_index, X)

    interpreter.invoke()
    preds = interpreter.get_tensor(output_index)
    return preds

In [103]:
pred = predict(x)
pred[0][0]

0.770491